In [3]:
import torch
x = torch.zeros([2,3])
print(x)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [142]:
import torch

def shift_to_mask(x_torch, dtype):
    x_shape = x_torch.shape
    
    dtype_filter    = torch.arange(10) < (17 - dtype)
    
    # [0xffff, 0xfffe, 0xfffc, 0xfff8, 0xfff0, 0xffe0, 0xffc0, 0xff80, 0xff00, 0xfe00]
    mask            = torch.tensor([-(1 << i) for i in range(10)]) * dtype_filter
    
    # [0x0000, 0x0001, 0x0002, 0x0004, 0x0008, 0x0010, 0x0020, 0x0040, 0x0080, 0x0100]
    offset          = torch.tensor([(256 >> (9 - i)) for i in range(10)]) * dtype_filter
    
    mask_flag = torch.zeros((10, x_shape[0], x_shape[1]), dtype=torch.int16)
    for idx in range(10):
        mask_flag[idx] = (torch.abs(x_torch) == (idx + 1))

    print(mask.reshape(-1,1,1))
    print("mask_flag",mask_flag, mask_flag.shape)
    
    mask_block = torch.sum(mask_flag * mask.reshape(-1,1,1), dim=0)
    offset_block = torch.sum(mask_flag * offset.reshape(-1,1,1), dim=0)
    print("mask_block",mask_block,mask_block.shape)

    return mask_block, offset_block
    
x_torch = torch.tensor([[9, 7, 0, 0],
                        [2, 4, 6, 8]], dtype=torch.int16) 
# dtype = 10 이므로, rolling이 경우의 수 7가지 이므로, 
# 7이상의 값은 0 나머지는 각각의 rolling 값에 따라0xff(-1) 0xfe(-2) 순서로 값을 가짐.
shift_to_mask(x_torch, 10)

tensor([[[ -1]],

        [[ -2]],

        [[ -4]],

        [[ -8]],

        [[-16]],

        [[-32]],

        [[-64]],

        [[  0]],

        [[  0]],

        [[  0]]])
mask_flag tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [1, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 1, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 1, 0]],

        [[0, 1, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 1]],

        [[1, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0]]], dtype=torch.int16) torch.Size([10, 2, 4])
mask_block tensor([[  0, -64,   0,   0],
        [ -2,  -8, -32,   0]]) torch.Size([2, 4])


(tensor([[  0, -64,   0,   0],
         [ -2,  -8, -32,   0]]),
 tensor([[ 0, 32,  0,  0],
         [ 1,  4, 16,  0]]))

In [42]:
import torch 

x = torch.arange(6).reshape(2,1,3,1)
y = torch.arange(10).reshape(1,2,1,5)

print(x * y)

# print((x * y).shape)

tensor([[[[ 0,  0,  0,  0,  0],
          [ 0,  1,  2,  3,  4],
          [ 0,  2,  4,  6,  8]],

         [[ 0,  0,  0,  0,  0],
          [ 5,  6,  7,  8,  9],
          [10, 12, 14, 16, 18]]],


        [[[ 0,  3,  6,  9, 12],
          [ 0,  4,  8, 12, 16],
          [ 0,  5, 10, 15, 20]],

         [[15, 18, 21, 24, 27],
          [20, 24, 28, 32, 36],
          [25, 30, 35, 40, 45]]]])


In [ ]:
import torch

def rolling_total(x_torch,length,dtype):
    x_shape = x_torch.shape

    rolling_flag = torch.zeros((10, x_shape[0], x_shape[1]), dtype=torch.int16)
    for idx in range(9):
        x_torch_value = (x_torch >= 2**(idx+6)) & (x_torch < 2**(idx+7)) | (x_torch < -(2**(idx+6))) & (x_torch >= -(2**(idx+7)))
        rolling_flag[idx] = ((x_torch_value) == True)  # rolling_value 가 1,-1d이면, idx=0에 저장.


In [53]:
import torch
import torch_dct as dct
a=torch.tensor([0,1,2,3])
b=a.type(torch.int16)
# c=a.int16()
print(torch.__version__)

SyntaxError: invalid syntax (2810875026.py, line 2)

In [185]:
import torch
import numpy as np
def display_per_block(x,length,value,select_block_index):
    x_numpy = x.numpy()
    x_shape = x_numpy.shape
    print(x_numpy)
    x_2D = x_numpy.reshape(-1,length)
    selected_value = x_2D[:,select_block_index]
    print("select value", selected_value, value)
    x_2D[:,select_block_index] = selected_value & value
    x_result = x_2D.reshape(x_shape)
    return x_result
     

x_torch = torch.tensor([[9, 7, 0, 9, 7, 0, 9, 7, 0, 9, 7, 0],
                        [2, 4, 8, 2, 4, 8, 2, 4, 8, 2, 4, 8],
                        [11, 12, 14, 11, 12, 14, 11, 12, 14, 11, 12, 14],
                        [5, 8, 25, 5, 8, 25, 5, 8, 25, 5, 8, 25]], dtype=torch.int16)
x_numpy = x_torch


one_hot =display_per_block(x_numpy,6,-4,3)
print(one_hot)

x_numpy [[ 9  7  0  9  7  0  9  7  0  9  7  0]
 [ 2  4  8  2  4  8  2  4  8  2  4  8]
 [11 12 14 11 12 14 11 12 14 11 12 14]
 [ 5  8 25  5  8 25  5  8 25  5  8 25]]
select value [ 9  9  2  2 11 11  5  5] -4
[[ 9  7  0  8  7  0  9  7  0  8  7  0]
 [ 2  4  8  0  4  8  2  4  8  0  4  8]
 [11 12 14  8 12 14 11 12 14  8 12 14]
 [ 5  8 25  4  8 25  5  8 25  4  8 25]]


In [ ]:
    # mask_default = [-1,0xffff,-1, ..., dtype = np.int64]
    # mask_default = np.array([ value for i in range(length)]) * block_filter       
    # mask_default_index_even = np.array([ value if i%2==0 else 0 for i in range(length)]) * block_filter
    # mask_default_index_odd  = np.array([ value if i%2==1 else 0 for i in range(length)]) * block_filter     
    # mask_incr = [0xffffffff, 0xfffffffe, ..., dtype = np.int64]
    # mask_incr = np.array([(value << i) for i in range(length)]) * block_filter  
    # mask_decr = np.array([(value << ((length-1)-i)) for i in range(length)]) * block_filter  
